In [ ]:
import numpy as np
import pandas as pd

fuzzy_cache = {}

df_car_sales = pd.read_csv('../dataset/vehicles.csv')
df_car_details = pd.read_json('../dataset/vehicle_details.json')

In [ ]:
# Pick a small sample of the data for testing
# Store processed data to prevent reload
sample_df_sales = df_car_sales.sample(n=10000).reset_index(drop=True)
# sample_df_sales = df_car_sales

In [ ]:
df_car_sales['type'].value_counts()

## #1 - Select relevant columns
We start by selecting only the relevant columns from the `sample_df_sales` and `df_details` dataframes. This step ensures that we focus on the necessary information for further processing and analysis. Irrelevant or redundant columns are dropped to streamline the workflow and reduce memory usage.

In [ ]:
# Data preprocessing
def select_columns(df_data, df_details):
    """
    Preprocess the sales and details dataframes by dropping irrelevant columns,
    removing rows with suspicious values, and normalizing the posting_date column.

    Parameters:
    df_sales (pd.DataFrame): The sales dataframe.
    df_details (pd.DataFrame): The details dataframe.

    Returns:
    pd.DataFrame, pd.DataFrame: The preprocessed sales and details dataframes.
    """
    # Select relevant columns
    df_data = df_data[['region', 'price', 'year', 'manufacturer', 'model', 
                         'condition', 'cylinders', 'fuel', 'odometer', 
                         'title_status', 'transmission', 'drive', 'size', 'type', 
                         'paint_color', 'state', 'posting_date']]
    # Useful details columns
    # df_details = df_details[['make', 'model', 'year', 'pv4', 'lv4', 
    #                      'displ', 'fuelcost08', 'yousavespend', 'fescore', 
    #                      'ghgscore', 'barrels08', 'co2tailpipegpm', 'vclass', 
    #                      'highway08', 'uhighway', 'comb08', 'ghgscorea', 'cylinders']]

    return df_data, df_details

## #2 - Fill the car details with actual data
The `sample_df_sales` may have missing data or information that doesn't match the model of the car. We can fill that information using `df_details`.

In [ ]:
from fuzzywuzzy import process
from tqdm import tqdm

# Use a dictionary to store previously seen matches

def get_matched_models(df_data, df_details):
    """
    Perform fuzzy matching to match makes and models between sample_df_sales and df_details.

    Parameters:
    sample_df_sales (pd.DataFrame): The sales dataframe containing manufacturer and model information.
    df_details (pd.DataFrame): The details dataframe containing make and model information.

    Returns:
    pd.DataFrame: The updated sample_df_sales dataframe with matched_make and matched_model columns.
    """

    def get_closest_match(row, column, choices):
        if pd.isnull(row[column]) or row[column] == '':
            return None
        value = row[column]
        if value in fuzzy_cache:
            return fuzzy_cache[value]
        match, score = process.extractOne(value, choices)
        result = match if score > 80 else None
        fuzzy_cache[value] = result
        return result

    # Apply fuzzy matching for manufacturer first
    df_details['make'] = df_details['make'].str.lower()
    
    # Add a progress bar to the process
    tqdm.pandas(desc="Matching manufacturers")
    df_data['matched_make'] = df_data.progress_apply(
        lambda row: get_closest_match(row, 'manufacturer', df_details['make'].unique()), axis=1
    )

    # Filter df_details to only include rows with the matched manufacturer
    def filter_models(row):
        if pd.isnull(row['matched_make']):
            return np.array([])  # Return an empty NumPy array
        return df_details[df_details['make'] == row['matched_make']]['model'].unique()

    # Apply fuzzy matching for model based on the filtered models
    def get_closest_model(row):
        models = filter_models(row)
        if models.size == 0:  # Explicitly check if the array is empty
            return None
        return get_closest_match(row, 'model', models)

    # Add a progress bar to the process
    tqdm.pandas(desc="Matching models")
    df_data['matched_model'] = df_data.progress_apply(get_closest_model, axis=1)

    # Calculate the number of matched and unmatched rows
    matched_count = df_data['matched_model'].notnull().sum()
    print(f"Matched: {matched_count} out of {len(df_data)} rows")

    return df_data

In [ ]:
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
    
def fill_model_deps(data_df, deps):
        """
        Fill missing values in model-dependent columns using the mode of data grouped by various keys.

        Parameters:
        df (pd.DataFrame): The dataframe to process.
        columns (list): List of model-dependent columns to fill.

        Returns:
        pd.DataFrame: The dataframe with missing values filled.
        """
        # Fill missing values grouped by manufacturer, model, and year
        for col in deps:
            data_df[col] = data_df.groupby(['manufacturer', 'model', 'year'])[col].transform(
                lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x
            )

        # Fill missing values grouped by manufacturer and model
        for col in deps:
            data_df[col] = data_df.groupby(['manufacturer', 'model'])[col].transform(
                lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x
            )

        # Fill missing values grouped by matched_make, matched_model, and year
        for col in deps:
            data_df[col] = data_df.groupby(['matched_make', 'matched_model', 'year'])[col].transform(
                lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x
            )

        # Fill missing values grouped by matched_make and matched_model
        for col in deps:
            data_df[col] = data_df.groupby(['matched_make', 'matched_model'])[col].transform(
                lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x
            )

        return data_df
    
def fill_non_deps(data_df, deps):
    # Define model-dependent columns

    # Select columns to impute (excluding model-dependent columns)
    columns_to_impute = [col for col in data_df.columns if col not in deps]

    # Impute missing values using K-Means clustering

    # Select numeric columns for clustering
    numeric_cols = data_df.select_dtypes(include=[np.number]).columns
    data_for_clustering = data_df[numeric_cols]

    # Impute missing values with the mean before clustering
    imputer = SimpleImputer(strategy='mean')
    data_imputed = imputer.fit_transform(data_for_clustering)

    # Apply K-Means clustering
    kmeans = KMeans(n_clusters=data_df['type'].nunique(), random_state=42)
    clusters = kmeans.fit_predict(data_imputed)

    # Add cluster labels to the dataframe
    data_df['cluster'] = clusters

    # Impute missing values in the original dataframe using cluster means
    for col in columns_to_impute:
        if data_df[col].dtype in [np.float64, np.int64]:  # Numeric columns
            data_df[col] = data_df.groupby('cluster')[col].transform(
                lambda x: x.fillna(x.mean())
            )
        elif data_df[col].dtype == object:  # Categorical columns
            data_df[col] = data_df.groupby('cluster')[col].transform(
                lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x
            )
            
    # Drop the cluster column
    data_df.drop(columns=['cluster'], inplace=True)
    
    return data_df


In [ ]:
def fill_details(df_data, df_details):
    """
    Fill missing values in target columns of df_data using corresponding source columns from df_details,
    and analyze categorical columns in the updated dataframe.

    Parameters:
    df_data (pd.DataFrame): The sales dataframe.
    df_details (pd.DataFrame): The details dataframe.
    columns_to_fill (dict): A dictionary where keys are target columns in df_data and values are source columns in df_details.

    Returns:
    pd.DataFrame: The updated dataframe with filled values.
    """
    columns_to_fill = {
        'cylinders': 'cylinders_y',
        'fuel': 'fueltype1',
        'transmission': 'trany',
        'type': 'vclass'
    }
    
    # Convert matching columns to lowercase for consistent comparison
    df_data['matched_make'] = df_data['matched_make'].str.lower()
    df_data['matched_model'] = df_data['matched_model'].str.lower()
    df_details['make'] = df_details['make'].str.lower()
    df_details['model'] = df_details['model'].str.lower()
    
    df_details.drop_duplicates(subset=['make', 'model', 'year'], inplace=True)
    
    # Merge the dataframes on relevant keys
    merged_df = pd.merge(df_data, df_details, how='left', left_on=['matched_make', 'matched_model', 'year'], right_on=['make','model','year'])[columns_to_fill.values()]

    # Fill missing values in target columns using corresponding source columns
    for target_col, source_col in columns_to_fill.items():
        df_data[target_col] = df_data[target_col].fillna(merged_df[source_col])

    return df_data

In [ ]:
def group_values(df):
    """
    Clean and preprocess specific columns in the dataframe.

    Parameters:
    df (pd.DataFrame): The dataframe to clean.

    Returns:
    pd.DataFrame: The cleaned dataframe.
    """
    # Clean the 'cylinders' column to keep only numeric values
    df['cylinders'] = df['cylinders'].str.extract(r'(\d+)').astype('Int64')
    df['transmission'] = df['transmission'].str.split(' ').str[0].str.lower()

    # Group the 'fuel' column into broader categories
    fuel_mapping = {
        'gas': 'gasoline',
        'Regular Gasoline': 'gasoline',
        'Premium Gasoline': 'gasoline',
        'Midgrade Gasoline': 'gasoline',
        'diesel': 'diesel',
        'Diesel': 'diesel',
        'electric': 'electricity',
        'Electricity': 'electricity',
        'hybrid': 'hybrid'
    }
    df['fuel'] = df['fuel'].map(fuel_mapping).fillna(df['fuel'])

    # Group the 'type' column into broader categories
    type_mapping = {
        'sedan': 'car',
        'coupe': 'car',
        'convertible': 'car',
        'hatchback': 'car',
        'wagon': 'car',
        'Midsize Cars': 'car',
        'Compact Cars': 'car',
        'Large Cars': 'car',
        'Subcompact Cars': 'car',
        'Minicompact Cars': 'car',
        'Two Seaters': 'car',
        'SUV': 'suv',
        'Sport Utility Vehicle - 4WD': 'suv',
        'Sport Utility Vehicle - 2WD': 'suv',
        'Small Sport Utility Vehicle 4WD': 'suv',
        'Small Sport Utility Vehicle 2WD': 'suv',
        'Standard Sport Utility Vehicle 4WD': 'suv',
        'Standard Sport Utility Vehicle 2WD': 'suv',
        'Special Purpose Vehicles': 'suv',
        'Special Purpose Vehicle 4WD': 'suv',
        'Special Purpose Vehicle 2WD': 'suv',
        'offroad': 'suv',
        'pickup': 'truck',
        'truck': 'truck',
        'Standard Pickup Trucks 4WD': 'truck',
        'Standard Pickup Trucks 2WD': 'truck',
        'Small Pickup Trucks 4WD': 'truck',
        'Small Pickup Trucks 2WD': 'truck',
        'Small Pickup Trucks': 'truck',
        'Standard Pickup Trucks': 'truck',
        'mini-van': 'van',
        'van': 'van',
        'bus': 'van',
        'Minivan - 2WD': 'van',
        'Minivan - 4WD': 'van',
        'Vans': 'van',
        'Vans, Cargo Type': 'van',
        'Vans, Passenger Type': 'van',
        'Small Station Wagons': 'car',
        'Midsize Station Wagons': 'car'
    }
    df['type'] = df['type'].map(type_mapping).fillna(df['type'])

    return df

In [ ]:

def clean_data(df_data, df_details):
    """
    Clean the sales and details dataframes by removing rows with missing values
    and resetting the index.

    Parameters:
    df_data (pd.DataFrame): The sales dataframe.
    df_details (pd.DataFrame): The details dataframe.

    Returns:
    pd.DataFrame, pd.DataFrame: The cleaned sales and details dataframes.
    """
    
    # Select relevant columns
    df_data, df_details = select_columns(df_data, df_details)
    
    # Replace 'other' values with NaN
    df_data.replace('other', np.nan, inplace=True)
    
    # Merge the dataframes to get more details
    df_data = get_matched_models(df_data, df_details)
    df_data = df_data[~df_data['matched_make'].isnull() & ~df_data['matched_model'].isnull()]
    
    # model_dependent_columns = ['cylinders', 'fuel', 'transmission', 'drive', 'size', 'type']
    model_dependent_columns = df_data
    
    # Fill model dependent values using df_details
    df_data = fill_details(df_data, df_details)
    df_data = group_values(df_data)

    # Fill model dependent values using modes of each model
    df_data = fill_model_deps(df_data, model_dependent_columns)
    # df_data = fill_non_deps(df_data, model_dependent_columns)
    
    df_data = df_data.dropna()
    
    return df_data

In [ ]:
df_clean = clean_data(sample_df_sales, df_car_details)
df_clean.info()

In [ ]:
df_clean.to_csv('../dataset/cleaned_sales_data.csv', index=False)